<a href="https://colab.research.google.com/github/DevOrtega/Pipeline-Kafka/blob/main/Mini_Pipeline_Big_Data_con_Kafka_%2B_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UT3 – Procesamiento de Datos Masivos y Gestión de la Información
## Actividad práctica: Mini-pipeline de Big Data con Kafka (RA3 – Sistemas de Big Data)
En este notebook vamos a construir un **mini-pipeline de datos** que combine:
1. Ingesta batch (CSV/JSON → DataFrame)
2. Ingesta en streaming usando Apache Kafka (productor + consumidor)
3. Limpieza y preprocesamiento
4. Modelado de datos (conceptual y lógico)
5. Evaluación de calidad e integridad
6. Consultas para extracción de valor
**Resultado de Aprendizaje trabajado (RA3 – SPS):**
> Gestiona y almacena datos facilitando la búsqueda de respuestas en grandes conjuntos de datos.
Autor/a: Grupo 6
Fecha: **(dd/mm/aaaa)**

In [ ]:
# @title Carga de librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import kagglehub
pd.set_option("display.max_columns", None)
sns.set(style="whitegrid")
print("Librerías de análisis de datos cargadas correctamente.")

Librerías de análisis de datos cargadas correctamente.


In [ ]:
# @title Instalación cliente Kafka y configuración de conexión
!pip install kafka-python --quiet
from kafka import KafkaProducer, KafkaConsumer
import json
import time

# ==============================================
# CONFIGURACIÓN DEL CLUSTER KAFKA (EXTERNO)
# ==============================================
# 1) Servidores bootstrap del cluster Kafka (proporcionados por el docente)
KAFKA_BOOTSTRAP_SERVERS = ["TU_BOOTSTRAP_SERVER:PORT"] # TODO: cambiar

# 2) Nombre del topic a utilizar para la práctica
KAFKA_TOPIC = "bigdata_ut3" # se puede personalizar por grupo, p.ej. "bigdata_ut3_grupo1"

# 3) ¿El cluster usa seguridad SASL_SSL?
USE_SASL_SSL = True # poner a False si el cluster NO usa SASL_SSL

# 4) Credenciales SASL/PLAIN (si aplica)
KAFKA_SECURITY_CONFIG = {
    "security_protocol": "SASL_SSL",
    "sasl_mechanism": "PLAIN",
    "sasl_plain_username": "TU_USERNAME_O_API_KEY", # TODO
    "sasl_plain_password": "TU_PASSWORD_O_API_SECRET" # TODO
}

print("Cliente Kafka preparado. Rellena los parámetros de conexión antes de usarlo.")

## 1. Descripción del dataset y metadatos

**Origen del dataset**: Kaggle

**Tipo de dato**: Estructurado (datos organizados en filas y columnas con atributos definidos)

**Metadatos básicos**:
- **Nombre del dataset**: Video Game Sales
- **Origen / URL**: https://www.kaggle.com/datasets/gregorut/videogamesales
- **Número aproximado de registros**: 16,598
- **Número de columnas**: 11
- **Temática**: Ventas de videojuegos (análisis de ventas por plataforma, región, género, etc.)
- **Formato**: CSV

**Descripción adicional**: Este dataset contiene información estructurada sobre las ventas de videojuegos en diferentes plataformas, incluyendo datos de ventas por región (NA, EU, JP, Other) y variables como nombre del juego, plataforma, año de lanzamiento, género, editor, entre otros.

In [ ]:
# @title 🧾 Metadatos del dataset (rellenar)
metadatos = {
    "nombre_dataset": "Video Game Sales",
    "origen": "Kaggle",
    "url_fuente": "https://www.kaggle.com/datasets/gregorut/videogamesales",
    "tipo_dato": "estructurado",
    "formato": "CSV",
    "descripcion": "Dataset con información sobre ventas de videojuegos en diferentes plataformas, incluyendo datos de ventas por región (NA, EU, JP, Other) y variables como nombre del juego, plataforma, año de lanzamiento, género, editor, entre otros. Contiene aproximadamente 16,598 registros y 11 columnas."
}
metadatos

In [ ]:
# Descargar el dataset - kagglehub devuelve la ruta donde se descargó
path = kagglehub.dataset_download("gregorut/videogamesales")

# Cargar el CSV (el archivo se llama vgsales.csv dentro del directorio descargado)
df = pd.read_csv(f"{path}/vgsales.csv")

# Opción B: Cargar desde archivo subido a Colab (descomentar si se usa)
# from google.colab import files
# uploaded = files.upload()
# df = pd.read_csv("vgsales.csv") # TODO

print("Dimensiones del dataset (filas, columnas):", df.shape)
df.head()

Using Colab cache for faster access to the 'videogamesales' dataset.
Dimensiones del dataset (filas, columnas): (16598, 11)


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [ ]:
# @title 🔍 Exploración inicial del dataset
print("Información general del dataset:")
df.info()
print("\nDescripción estadística de variables numéricas:")
display(df.describe())
print("\nPrimeras filas:")
display(df.head())
print("\nNúmero de valores nulos por columna:")
display(df.isnull().sum())

Información general del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB

Descripción estadística de variables numéricas:


,Rank,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
count,16598.000000,16327.000000,16598.000000,16598.000000,16598.000000,16598.000000,16598.000000
mean,8300.605254,2006.406443,0.264667,0.146652,0.077782,0.048063,0.537441
std,4791.853933,5.828981,0.816683,0.505351,0.309291,0.188588,1.555028
min,1.000000,1980.000000,0.000000,0.000000,0.000000,0.000000,0.010000
25%,4151.250000,2003.000000,0.000000,0.000000,0.000000,0.000000,0.060000
50%,8300.500000,2007.000000,0.080000,0.020000,0.000000,0.010000,0.170000
75%,12449.750000,2010.000000,0.240000,0.110000,0.040000,0.040000,0.470000
max,16600.000000,2020.000000,41.490000,29.020000,10.220000,10.570000,82.740000



Primeras filas:


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37



Número de valores nulos por columna:


,0
Rank,0
Name,0
Platform,0
Year,271
Genre,0
Publisher,58
NA_Sales,0
EU_Sales,0
JP_Sales,0
Other_Sales,0


## 2. Objetivos de negocio y preguntas clave
Define **al menos 3 preguntas de negocio** que quieras responder con este dataset.
Ejemplos:
- ¿Qué factores influyen más en X?
- ¿Qué tipo de clientes generan más ingresos?
- ¿Qué días/horas se concentra más actividad?
> Escribe aquí tus preguntas:
1. …
2. …
3. …

In [ ]:
# @title Productor Kafka: envío del dataset fila a fila
# Configuración del productor
producer_config = {
    "bootstrap_servers": KAFKA_BOOTSTRAP_SERVERS,
    "value_serializer": lambda v: json.dumps(v).encode("utf-8")
}

if USE_SASL_SSL:
    producer_config.update(KAFKA_SECURITY_CONFIG)

producer = KafkaProducer(**producer_config)

# Convertir DataFrame a lista de diccionarios (cada fila = mensaje)
registros = df.to_dict(orient="records")
print(f"Enviando {len(registros)} mensajes al topic '{KAFKA_TOPIC}'...")

for i, fila in enumerate(registros):
    producer.send(KAFKA_TOPIC, value=fila)
    # Retardo opcional para simular streaming
    # time.sleep(0.01)

producer.flush()
print("✅ Envío completado.")

In [ ]:
# @title Consumidor Kafka: lectura y reconstrucción del DataFrame

# Configuración básica
consumer_config = {
    "bootstrap_servers": KAFKA_BOOTSTRAP_SERVERS,
    "auto_offset_reset": "earliest",  # empezamos desde el inicio del topic
    "enable_auto_commit": True,
    "group_id": "grupo_ut3",          # TODO: se puede personalizar por grupo
    "value_deserializer": lambda m: json.loads(m.decode("utf-8"))
}

# Aplicar seguridad si está activada
if USE_SASL_SSL:
    consumer_config.update(KAFKA_SECURITY_CONFIG)

# Instancia del consumidor
consumer = KafkaConsumer(KAFKA_TOPIC, **consumer_config)

registros_consumidos = []

# IMPORTANTE: 'df' debe existir en memoria (de una celda anterior de carga de datos)
MAX_MENSAJES = df.shape[0]

print("Leyendo mensajes del topic...")

# Bucle de consumo
for msg in consumer:
    registros_consumidos.append(msg.value)

    # Condición de parada
    if len(registros_consumidos) >= MAX_MENSAJES:
        break

print(f" Mensajes leídos desde Kafka: {len(registros_consumidos)}")

# Crear DataFrame y mostrar resultados
df_stream = pd.DataFrame(registros_consumidos)
display(df_stream.head())
print("Dimensiones de df_stream:", df_stream.shape)

In [ ]:
# @title Limpieza básica sobre df_stream
df_clean = df_stream.copy()
print("Registros duplicados antes de limpiar:", df_clean.duplicated().sum())
df_clean = df_clean.drop_duplicates()
print("Registros duplicados después de limpiar:", df_clean.duplicated().sum())
print("\nValores nulos por columna (antes de imputar):")
display(df_clean.isnull().sum())
# TODO: definir estrategia de imputación según el dataset
# Ejemplo para columnas numéricas: imputar con la mediana
numeric_cols_tmp = df_clean.select_dtypes(include=["int64", "float64"]).columns
df_clean[numeric_cols_tmp] = df_clean[numeric_cols_tmp].fillna(df_clean[numeric_cols_tmp].median())
print("\nValores nulos por columna (después de imputar numéricas):")
display(df_clean.isnull().sum())
df_clean.head()

In [ ]:
# @title Boxplots para detección de outliers (opcional)
numeric_cols = df_clean.select_dtypes(include=["int64", "float64"]).columns
print("Columnas numéricas:", list(numeric_cols))

for col in numeric_cols:
    plt.figure(figsize=(6, 3))
    sns.boxplot(x=df_clean[col])
    plt.title(f"Boxplot de {col}")
    plt.show()

# TODO: decidir si se filtran o transforman outliers en alguna columna

## 3. Modelado de datos (conceptual y lógico)
En esta sección diseña:
1. **Modelo conceptual**
- Entidades principales (ej.: Cliente, Pedido, Producto, etc.)
- Relaciones entre entidades (ej.: Cliente — realiza — Pedido)
2. **Modelo lógico**
- Elige el tipo de modelo de almacenamiento:
- Relacional (tablas SQL)
- NoSQL documental (MongoDB)
- Columnar (Cassandra)
- Clave-valor (Redis)
- Grafos (Neo4j)
- Justifica la elección:
- Tipo de consultas
- Volumen y variedad de los datos
- Necesidades de escalabilidad / rendimiento
> Adjunta o describe aquí tu modelo (texto, tabla resumen o enlace a un diagrama).

In [ ]:
# @title Selección de variables y pipeline de preprocesamiento
df_model = df_clean.copy()

# TODO: definir variable objetivo (si procede). Si no hay objetivo, dejar a None.
target = None # p.ej.: "tip" en el dataset 'tips'

if target is not None and target in df_model.columns:
    features = [col for col in df_model.columns if col != target]
    X = df_model[features].copy()
    y = df_model[target].copy()
else:
    features = list(df_model.columns)
    X = df_model.copy()
    y = None

print("Features:", features)
print("Target:", target if target is not None and target in df_model.columns else "No se ha definido variable objetivo")

numeric_features = X.select_dtypes(include=["int64", "float64"]).columns
categorical_features = X.select_dtypes(include=["object", "category"]).columns

print("\nColumnas numéricas:", list(numeric_features))
print("Columnas categóricas:", list(categorical_features))

# Transformador numérico: escalado MinMax
numeric_transformer = Pipeline(steps=[
    ("scaler", MinMaxScaler())
])

# Transformador categórico: One-Hot Encoding
categorical_transformer = Pipeline(steps=[
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric

In [ ]:
# @title Transformación y guardado de datos procesados
X_train_transformed = preprocessor.transform(X_train)
print("Shape de X_train transformado:", X_train_transformed.shape)

# Intento convertir a DataFrame (puede ser grande)
try:
    X_train_dense = pd.DataFrame(
        X_train_transformed.toarray() if hasattr(X_train_transformed, "toarray") else X_train_transformed
    )
    display(X_train_dense.head())
except Exception as e:
    print("No se ha podido convertir a DataFrame por tamaño/formato:", e)

# Guardar conjunto limpio (sin codificación) como ejemplo
df_clean.to_csv("datos_procesados.csv", index=False)
df_clean.to_parquet("datos_procesados.parquet", index=False)
print("Archivos 'datos_procesados.csv' y 'datos_procesados.parquet' guardados.")

## 4. Calidad del dato e integridad
Evalúa la calidad de tu dataset **después del preprocesamiento**:
- **Exactitud**: ¿Los datos parecen razonables / realistas?
- **Completitud**: ¿Qué porcentaje de nulos queda? ¿En qué columnas?
- **Consistencia**: ¿Hay valores contradictorios o incoherentes?
- **Unicidad**: ¿Persisten duplicados?
- **Oportunidad**: ¿Los datos siguen siendo válidos / actuales para el problema?
> Escribe aquí un breve análisis de calidad del dato y las decisiones tomadas en tu pipeline.

In [ ]:
# @title Métricas simples de calidad del dato
n_filas, n_cols = df_clean.shape
porcentaje_nulos = (df_clean.isnull().sum() / n_filas) * 100
duplicados = df_clean.duplicated().sum()
print("Número de filas:", n_filas)
print("Número de columnas:", n_cols)
print("\nPorcentaje de nulos por columna:")
display(porcentaje_nulos)
print("\nNúmero de registros duplicados:", duplicados)

In [ ]:
# @title Consultas de negocio y agregaciones
# TODO: adaptar estas consultas a tu dataset concreto
print("Columnas disponibles en df_clean:")
print(df_clean.columns)

# Ejemplo genérico 1: agrupar por una columna categórica
col_cat_ejemplo = None # p.ej. "sex", "day", "species", etc.
col_num_ejemplo = None # p.ej. "total_bill", "tip", "sepal_length", etc.

if col_cat_ejemplo in df_clean.columns and col_num_ejemplo in df_clean.columns:
    print(f"\nMedia de {col_num_ejemplo} por {col_cat_ejemplo}:")
    display(df_clean.groupby(col_cat_ejemplo)[col_num_ejemplo].mean())
else:
    print("\nConfigura 'col_cat_ejemplo' y 'col_num_ejemplo' según tu dataset para ver un ejemplo.")

# Añade aquí tus propias consultas que respondan a las preguntas de negocio
# Ejemplo:
# resultado_1 = df_clean.groupby("ALGUNA_COLUMNA")["OTRA_COLUMNA"].agg(["mean", "sum"])
# display(resultado_1)

## 5. Conclusiones del mini-pipeline
1. **Resumen del proceso**
- Describe brevemente las etapas: ingesta batch, envío a Kafka, consumo desde Kafka,
limpieza, modelado, calidad del dato y consultas.
2. **Respuestas a las preguntas de negocio**
- ¿Qué has podido responder?
- ¿Qué patrones o insights relevantes has obtenido?
3. **Limitaciones y mejoras**
- ¿Qué harías mejor si tuvieras más tiempo o más herramientas (Spark, MongoDB, etc.)?
- ¿Qué otras fuentes de datos podrían enriquecer el análisis?
4. **Relación con el RA3 de Sistemas de Big Data**
- Explica en 4–5 líneas cómo este mini-pipeline te ha permitido:
- gestionar datos,
- almacenarlos de forma apropiada,
- y facilitar la búsqueda de respuestas en grandes conjuntos de datos.